# Create a Custom Loss

In this tutorial, we will go over `Loss` objects, how they can be declared and assigned, and how to build a custom `Loss`.

Let's start by importing what we need.

In [6]:
import os
import sys
import numpy as np
import tensorflow as tf

colab_env = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
colab_initialized = True if os.path.exists("MotorNet") else False

if colab_env and not colab_initialized:
  !git clone https://github.com/OlivierCodol/MotorNet
  sys.path.append('MotorNet')
  print("Running cell using COLAB initialization...")
elif colab_env and colab_initialized:
  print("Already initialized using COLAB initialization.")
else:
  paths = [p for p in sys.path if os.path.exists(p)]
  local_initialized = True if [p for p in paths if "motornet" in os.listdir(p)] else False
  if local_initialized:
    %load_ext autoreload
    %autoreload 2
    print("Already initialized using LOCAL initialization.")
  else:
    path = [p for p in paths if p.__contains__("tutorials")]
    if len(path) != 1:
      raise ValueError("Path to MotorNet could not be determined with certainty.")
    sys.path.append(os.path.dirname(path[:path.rfind('tutorials')]))
    %load_ext autoreload
    %autoreload 2
    print("Running cell using LOCAL initialization...")


import motornet as mn

print('All packages imported.')
print('tensorflow version: ' + tf.__version__)
print('numpy version: ' + np.__version__)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Already initialized using LOCAL initialization.
All packages imported.
tensorflow version: 2.9.1
numpy version: 1.22.4


# I. Losses in Task Objects

## I. 1. Printing out currently declared losses

The simplest way to handle losses is via `Task` objects. Note that declaring losses via `Task` objects is not the only viable way, but this is easier, as otherwise we would have to manually do all the things that `add_loss` does automatically for us. Also, this may result in misleading (shuffled) loss labels when printing progress bars at runtime because `tensorflow` models (`tf.keras.Model`) do not maintain loss label orders properly for some reason. Adding losses to the task object will make it available to our curstom-made `tf.keras.Model` subclass, which is `mn.nets.MotorNetModel`. In `MotorNetModel` instances, the losses available in `Task` subclasses will be re-ordered properly at initialization to avoid the parent class `tf.keras.Model` reshuffling the labels in a wrong order.

Here, we import a pre-built `Task` object called `CentreOutReach`. Some losses are already included by default and we can print them out using `print_losses`.


In [7]:

plant = mn.plants.ReluPointMass24()
network = mn.nets.layers.GRUNetwork(plant=plant, n_units=50, kernel_regularizer=10**-6, name='network')

task = mn.tasks.CentreOutReach(network=network)

task.print_losses()


ASSIGNED OUTPUT: cartesian position
-----------------------------------
loss function:  <motornet.nets.losses.PositionLoss object at 0x0000012A9CA5B160>
loss weight:    1.0
loss name:      position
Compounded:     NO


ASSIGNED OUTPUT: muscle state
-----------------------------
loss function:  <motornet.nets.losses.L2xDxActivationLoss object at 0x0000012A9CA5B0D0>
loss weight:    5
loss name:      l2_xdx_activation
Compounded:     NO


ASSIGNED OUTPUT: gru_hidden_0
-----------------------------
loss function:  <motornet.nets.losses.L2xDxRegularizer object at 0x0000012A9CA5B070>
loss weight:    0.1
loss name:      gru_regularizer
Compounded:     NO





## I. 2. Declaring new losses via the Task object

Losses can be declared via the `Task` object using the `add_loss` method. Feel free to check the Reference Manual online for more details on that method, but this is briefly reproduced here for convenience. See below the arguments the `add_loss` method can take.

- `loss`: A `tensorflow.python.keras.losses.Loss` object class or subclass. `Loss` subclasses specific to `MotorNet` are also available in the `motornet.nets.losses` module.

- `assigned_output`: A string indicating the output state that the loss will be applied to. This should correspond to an output name from the `Network` object instance passed at initialization. The output names can be retrieved via the `motornet.nets.layers.Network.output_names` attribute.

- `loss_weight`: [Optional], A float indicating the weight of the loss when all contributing losses are added to the total loss. Default: is `1.0`.

- `name`: [Optional], A string indicating the name (label) to give to the loss object. This is used to print, plot, and save losses during training.

If we add a loss using the `add_loss` method and print the losses again, we can see that the new loss is now included in the `Task` object.


In [8]:

task.add_loss(loss=mn.nets.losses.PositionLoss(), assigned_output="cartesian position")
task.print_losses()


ASSIGNED OUTPUT: cartesian position
-----------------------------------
loss function:  <motornet.nets.losses.PositionLoss object at 0x0000012A9CA5B160>
loss weight:    1.0
loss name:      position
Compounded:     YES


ASSIGNED OUTPUT: cartesian position
-----------------------------------
loss function:  <motornet.nets.losses.PositionLoss object at 0x0000012A958A8A00>
loss weight:    1.0
loss name:      position
Compounded:     YES


ASSIGNED OUTPUT: muscle state
-----------------------------
loss function:  <motornet.nets.losses.L2xDxActivationLoss object at 0x0000012A9CA5B0D0>
loss weight:    5
loss name:      l2_xdx_activation
Compounded:     NO


ASSIGNED OUTPUT: gru_hidden_0
-----------------------------
loss function:  <motornet.nets.losses.L2xDxRegularizer object at 0x0000012A9CA5B070>
loss weight:    0.1
loss name:      gru_regularizer
Compounded:     NO




Note that "Compounded" indicates if the loss shares an assigned output with another loss. This is because several losses can be assigned to the same output state. If this is the case, then their loss value will be added together, weighted by the loss weight of each contributing loss.


# II. Creating a Custom Loss Object

The Loss objects passed to the `loss`argument in the `add_loss` methods are subclasses of `tensorflow.python.keras.losses.LossFunctionWrapper` objects from TensorFlow.

First, we need a loss function, which must take at least a `y_true` and `y_pred` input, in that order. Note that those two arguments must be present even if they are not used. Extra arguments may then be passed as well after this. This function must contain the loss formula leading to the penalty used for training the network, and return said penalty.

When calling the base class at initialization, the loss function must be specified as the first input, followed by optional arguments such as the name of the function or the reduction method. Finally, extra arguments to be passed the loss function must be passed last.

For more details about the reduction methods to use, feel free to check this custom training [tutorial](https://www.tensorflow.org/tutorials/distribute/custom_training) from TensorFlow.
For more details on how to subclass `LossFunctionWrapper` objects, feel free to check the TensorFlow documentation.

In [9]:

# importing dependencies
from tensorflow.python.keras.losses import LossFunctionWrapper
from tensorflow.python.keras.utils import losses_utils


# creating loss function
def _l2_activation_loss(y_true, y_pred, extra_arg_1, extra_arg_2):
    activation = tf.slice(y_pred, [0, 0, 0, 0], [-1, -1, 1, -1])
    return extra_arg_1 * tf.reduce_mean(activation ** 2) + extra_arg_2


# creating loss subclass
class L2ActivationLoss(LossFunctionWrapper):

    def __init__(self, extra_arg_1, extra_arg_2=1, name='l2_activation', reduction=losses_utils.ReductionV2.AUTO):

        super().__init__(_l2_activation_loss, name=name, reduction=reduction, extra_arg_1=extra_arg_1, extra_arg_2=extra_arg_2)

        # one can add the extra arguments passed as attributes if desired
        self.extra_arg_1 = extra_arg_1
        self.extra_arg_2 = extra_arg_2


# creating loss instance
new_loss_object = L2ActivationLoss(extra_arg_1=1, name='example_loss')



We can now assign our newly-made custom loss to a `Network` state of our choosing using the process described in section I.2. above. For instance, for the `excitation` state:


In [10]:

task.add_loss(loss=new_loss_object, assigned_output='excitation')
task.print_losses()


ASSIGNED OUTPUT: cartesian position
-----------------------------------
loss function:  <motornet.nets.losses.PositionLoss object at 0x0000012A9CA5B160>
loss weight:    1.0
loss name:      position
Compounded:     YES


ASSIGNED OUTPUT: cartesian position
-----------------------------------
loss function:  <motornet.nets.losses.PositionLoss object at 0x0000012A958A8A00>
loss weight:    1.0
loss name:      position
Compounded:     YES


ASSIGNED OUTPUT: muscle state
-----------------------------
loss function:  <motornet.nets.losses.L2xDxActivationLoss object at 0x0000012A9CA5B0D0>
loss weight:    5
loss name:      l2_xdx_activation
Compounded:     NO


ASSIGNED OUTPUT: excitation
---------------------------
loss function:  <__main__.L2ActivationLoss object at 0x0000012A9CA5BC10>
loss weight:    1.0
loss name:      example_loss
Compounded:     NO


ASSIGNED OUTPUT: gru_hidden_0
-----------------------------
loss function:  <motornet.nets.losses.L2xDxRegularizer object at 0x0000012A9CA5B